<a href="https://colab.research.google.com/github/norflin321/ml/blob/main/nano_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
from torch.nn import functional as fun

with open("tiny_shakespeare.txt", "r", encoding="utf-8") as file:
    text = file.read()

print("-- dataset len:", len(text))

# here are all the unique characters that occur in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])
print(f"-- dataset has {vocab_size} unique characters:", itos)

# let's now encode the entire text dataset and store it into a torch.tensor
data = torch.tensor(encode(text), dtype=torch.long)

# let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

inputs, targets = get_batch("train")
print("-- inputs:", inputs)
print("-- targets:", targets)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = inputs[b, :t+1]
        target = targets[b, t]
        print(f"-- when input is {context.tolist()} the target: {target}")

torch.manual_seed(1337)
class BigramLanguageModel(torch.nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # create embeding table which is a tensor of shape [64, 64] filled with random numbers
        self.token_embedding_table = torch.nn.Embedding(vocab_size, vocab_size)
    def forward(self, inputs, targets):
        # we cant just train a model on indices of characters from lookup table, before we need to embed them.
        # after embeding we should get a tensor with shape [4, 8, 64], which means that each of 8 numbers in 4 rows, now represented as one dim tensor with 64 random numbers
        logits = self.token_embedding_table(inputs)
        return logits

model = BigramLanguageModel(vocab_size) # initializa the model
logits = model(inputs, targets) # forward pass

print("-- logits:", logits.shape)

-- dataset len: 1115394
-- dataset has 65 unique characters: {0: '\n', 1: ' ', 2: '!', 3: '$', 4: '&', 5: "'", 6: ',', 7: '-', 8: '.', 9: '3', 10: ':', 11: ';', 12: '?', 13: 'A', 14: 'B', 15: 'C', 16: 'D', 17: 'E', 18: 'F', 19: 'G', 20: 'H', 21: 'I', 22: 'J', 23: 'K', 24: 'L', 25: 'M', 26: 'N', 27: 'O', 28: 'P', 29: 'Q', 30: 'R', 31: 'S', 32: 'T', 33: 'U', 34: 'V', 35: 'W', 36: 'X', 37: 'Y', 38: 'Z', 39: 'a', 40: 'b', 41: 'c', 42: 'd', 43: 'e', 44: 'f', 45: 'g', 46: 'h', 47: 'i', 48: 'j', 49: 'k', 50: 'l', 51: 'm', 52: 'n', 53: 'o', 54: 'p', 55: 'q', 56: 'r', 57: 's', 58: 't', 59: 'u', 60: 'v', 61: 'w', 62: 'x', 63: 'y', 64: 'z'}
-- inputs: tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
-- targets: tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-

# Новый раздел